# Loading the Nodes

In [1]:
import numpy as np
import pandas as pd
import json
from tqdm.notebook import tqdm
import gc
from scipy import sparse, io
from scipy.special import softmax
import os

from collections import deque

In [2]:
df=pd.read_parquet("/kaggle/input/dataset-with-embeddings/cs_papers_wo_embeddings.parquet")
df['categories'] = df['categories'].apply(lambda x: x.replace(', ', ''))
df.head()

,id,authors,title,categories,abstract,update_date,degree,num_citations,num_references
0,0704.0046,"I. Csiszar, F. Hiai, D. Petz",A limit relation for entropy and channel capac...,quant-ph cs.IT math.IT,"In a quantum mechanical model, Diosi, Feldma...",2009-11-13,1,1,0
1,0704.0062,"Rastislav \v{S}r\'amek, Bro\v{n}a Brejov\'a, T...",On-line Viterbi Algorithm and Its Relationship...,cs.DS,"In this paper, we introduce the on-line Vite...",2010-01-25,2,2,0
2,0704.0098,"Jack Raymond, David Saad",Sparsely-spread CDMA - a statistical mechanics...,cs.IT math.IT,"Sparse Code Division Multiple Access (CDMA),...",2009-11-13,1,1,0
3,0704.0108,Sergey Gubin,Reducing SAT to 2-SAT,cs.CC,Description of a polynomial time reduction o...,2007-05-23,5,1,4
4,0704.0213,Ketan D. Mulmuley Hariharan Narayanan,Geometric Complexity Theory V: On deciding non...,cs.CC,This article has been withdrawn because it h...,2012-09-28,1,1,0


## Utility Functions to make sure that only valid references are added

In [3]:
def get_paper_release_date_and_serial_no(paper_id):
    yymm = None
    try:
        float(paper_id)
        yymm = paper_id.split('.')[0]
        serial_no = paper_id.split('.')[1]
    except ValueError:
        yymm = paper_id.split('/')[-1][:4]
        serial_no = paper_id.split('/')[-1][4:]

    if yymm.startswith('9'):
        paper_yyyy = '19' + yymm[:2]
    else:
        paper_yyyy = '20' + yymm[:2]
    paper_mm = yymm[-2:]
        
    return paper_yyyy + paper_mm, serial_no

def paper_1_came_before_paper_2(paper1_id, paper2_id):
    paper1_yyyymm, paper1_sr_no = get_paper_release_date_and_serial_no(paper1_id)
    paper2_yyyymm, paper2_sr_no = get_paper_release_date_and_serial_no(paper2_id)
    
    try:
        if int(paper1_yyyymm) == int(paper2_yyyymm):
            return paper1_sr_no < paper1_sr_no
    except ValueError:
        print(paper1_id, paper2_id)
        print(paper1_yyyymm, paper2_yyyymm)
        print(paper1_sr_no, paper2_sr_no)
        
        raise ValueError
    
    
    return paper1_yyyymm < paper2_yyyymm
    

In [4]:
class Node:
    def __init__(self, node_id, authors=None, title=None, categories=None, abstract=None, embeddings=None, update_date=None, degree=None, num_citations=None, num_references=None):
        self.id = str(node_id)
        self.authors = self.parse_authors(authors)
        self.title = self.remove_newlines(title)
        self.categories = categories if categories else []
        self.abstract = self.remove_newlines(abstract)
        self.update_date = update_date
        self.embeddings = embeddings
        
        self.degree = degree if degree is not None else 0
        self.num_citations = num_citations if num_citations is not None else 0
        self.num_references = num_references if num_references is not None else 0

    # function to store authors in list format
    def parse_authors(self, authors_string):
        if authors_string:
            authors_list = []
            for author in authors_string.split(" and "):
                authors_list.extend(author.split(", "))
            return authors_list
        else:
            return []

    # function to replace newline characters ("\n") with an empty string
    def remove_newlines(self, text):
        return text.replace("\n", " ")
    
    def get_title(self):
        return self.title
    
    def get_id(self):
        return self.id
    
    def get_info(self):
        info = {   
            'id': self.id,
            'authors': ', '.join(self.authors),
            'title': self.title,
            'categories': self.categories,
            'abstract': self.abstract,
            'embedding': self.embeddings,
            'update_date': self.update_date,
            'degree': self.degree,
            'num_citations': self.num_citations,
            'num_references': self.num_references
        }
        
        return info
    
    def update_degrees(self, degree, citations, references):
        self.degree = degree
        self.num_citations = citations
        self.num_references = references
    
class Graph:
    def __init__(self):
        self.nodes = {}
        self.undirected_edges = {}
        self.citation_edges = {}
        self.reference_edges = {}
        
        self.node_degrees = {}
        self.num_citations = {}
        self.num_references = {}
        
        self.v_to_i = {}

    # function to add nodes in the Graph data structure
    def add_node(self, node_id, authors=None, title=None, categories=None, abstract=None, embedding=None, update_date=None, degree=None, num_citations=None, num_references=None):
        node_id = str(node_id)
        if node_id not in self.nodes:
            self.nodes[node_id] = Node(node_id, authors, title, categories, abstract, embedding, update_date, degree, num_citations, num_references)

    # function to add edges in the Graph data structure
    def add_edge(self, node1, node2):
        node1, node2 = str(node1), str(node2)
        if paper_1_came_before_paper_2(node2, node1):
            if node1 in self.nodes and node2 in self.nodes:
                # conditions to add nodes to the edge lists of themselves and the neighbour node
                if node1 not in self.undirected_edges:
                    self.undirected_edges[node1] = []
                    self.node_degrees[node1] = 0
                if node2 not in self.undirected_edges:
                    self.undirected_edges[node2] = []
                    self.node_degrees[node2] = 0
                if node2 not in self.undirected_edges[node1]:
                    self.undirected_edges[node1].append(node2)
                    self.node_degrees[node1] += 1
                if node1 not in self.undirected_edges[node2]:
                    self.undirected_edges[node2].append(node1)
                    self.node_degrees[node2] += 1
                
                # when node1 references node2 i.e. referring
                if node1 not in self.reference_edges:
                    self.reference_edges[node1] = []
                    self.num_references[node1] = 0
                if node2 not in self.reference_edges[node1]:
                    self.reference_edges[node1].append(node2)
                    self.num_references[node1] += 1
            
                
                # when node2 is referred by node1 i.e. citation
                if node2 not in self.citation_edges:
                    self.citation_edges[node2] = []
                    self.num_citations[node2] = 0
                if node1 not in self.citation_edges[node2]:
                    self.citation_edges[node2].append(node1)
                    self.num_citations[node2] += 1
                
    # update the degrees for all the nodes
    def update_degrees(self):
        for node_id, node_obj in tqdm(self.node_degrees.items(), total=len(list(self.node_degrees))):
            try:
                self.nodes[node_id].update_degrees(degree=self.node_degrees[node_id], citations=self.num_citations.get(node_id, 0), references=self.num_references.get(node_id, 0))
            except KeyError as e:
                raise KeyError(f"KeyError occurred: {e}")

    # function to return list of nodes present in graph
    def get_nodes(self):
        return list(self.nodes)
    
    # get adjacency matrix based on the reference edges
    def get_adjacency_matrix(self):
        self.v_to_i = {}
        for i, node_id in enumerate(list(self.nodes.keys())):
            self.v_to_i[node_id] = i
        
        matrix = np.zeros((len(self.nodes.keys()), len(self.nodes.keys())), dtype=np.float16)
        
        for node_id in list(self.reference_edges.keys()):
            adjacent_nodes = self.reference_edges[node_id]
            
            for adj_node in adjacent_nodes:
                matrix[self.v_to_i[node_id], self.v_to_i[adj_node]] = 1

        gc.collect()
        
        return matrix
    
    # get the adjacency matrix based of the citation edges
    def get_adjacency_matrix_citation(self):
        self.v_to_i = {}
        for i, node_id in enumerate(list(self.nodes.keys())):
            self.v_to_i[node_id] = i
        
        matrix = np.zeros((len(self.nodes.keys()), len(self.nodes.keys())), dtype=np.float16)
        
        for node_id in list(self.citation_edges.keys()):
            adjacent_nodes = self.citation_edges[node_id]
            
            for adj_node in adjacent_nodes:
                matrix[self.v_to_i[node_id], self.v_to_i[adj_node]] = 1

        gc.collect()
        
        return matrix
    
    # return all the edges in the adjacency list format
    def get_adjacency_list(self):
        return self.undirected_edges, self.citation_edges, self.reference_edges
    
    # given the title of the paper, return the node
    def get_node_by_title(self, title):
        """
        Return the node corresponding to the research paper with the given title.
        If not found, return None.
        """
        
        for node in self.nodes:
            # print(node.id,title)
            if self.nodes[node].get_title()==title:
                return self.nodes[node].get_info()
        return None
    
    # utility function to remove all the nodes with zero degree
    def remove_isolated_nodes(self):
        node_ids = list(self.nodes.keys())
        for id in tqdm(node_ids):
            if self.nodes[id].degree == 0:
                del self.nodes[id]

    # function to return list of edges present in graph based on the undirected edges
    def get_edge_list(self):
        edge_list = []
        for node, connected_nodes in tqdm(self.undirected_edges.items(), total=len(list(self.undirected_edges.keys()))):
            for connected_node in connected_nodes:
                edge_list.append((node, connected_node))
        return edge_list


In [5]:
graph = Graph()

# make the graph based on the node dataframe
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    node_id = str(row['id'])
    authors = row['authors']
    title = row['title']
    categories = row['categories']
    abstract = row['abstract']
    embedding = None if 'embeddings' not in df.columns else row['embeddings']
    update_date = row['update_date']
    degree = row['degree']
    num_citations = row['num_citations']
    num_references = row['num_references']
    

    graph.add_node(node_id, authors=authors, title=title, categories=categories, abstract=abstract, embedding=embedding, update_date=update_date, degree=degree, num_citations=num_citations, num_references=num_references)

n_nodes = len(graph.nodes)
gc.collect()

  0%|          | 0/88281 [00:00<?, ?it/s]

19

# Creating the Graph

In [6]:
# load the reference edges
citation_data = pd.read_parquet("/kaggle/input/dataset-with-embeddings/reference_edges.parquet")

In [7]:
# # to add self edges
# paper_ids = list(citation_data["id"])
# for i in tqdm(range(citation_data.shape[0])):
#     refer_list = list(citation_data.loc[i, 'refers'])
#     refer_list.append(citation_data.loc[i, 'id'])
#     citation_data.loc[i, 'refers'] = np.array(refer_list)
#     break

In [8]:
# add edges to the graph
for i, row in tqdm(citation_data.iterrows(), total=citation_data.shape[0]):
    source_id = row['paper_id']
    target_ids = row['refers_to']
    for target_id in target_ids: 
        graph.add_edge(source_id, target_id)
        
gc.collect()

  0%|          | 0/66877 [00:00<?, ?it/s]

19

In [9]:
adj_mat = graph.get_adjacency_matrix()
adj_mat.shape

(88281, 88281)

In [10]:
def pagerank(adjacency, probs=None, n_iters=20, resid=0.4):
    """
    Args:
        adjacency - sparse matrix
        probs - array of initial random jump probabilities.
    """
    n_nodes, _ = adjacency.shape
    adjacency = sparse.lil_array(adjacency.astype(bool)).astype(float)
    # set 'sinks' to have random transitions
    out_degrees = adjacency.sum(1)
    transitions = adjacency*(1/out_degrees[:, np.newaxis])
    transitions = transitions.T
    transitions = sparse.csc_array(transitions)
    # initial rank: all equal
    # adjacency[i, j] indicates an edge from i to j 
    if probs is None:
        probs = np.ones(n_nodes)/n_nodes
        probs = probs[:, np.newaxis]
    # run power iterations
    for _ in tqdm(range(n_iters)):
        probs = resid*transitions.dot(probs) + (1 - resid)*(1/n_nodes)
        probs /= probs.sum()
        
    # return the stationary probability, and the transition matrix
    return probs, transitions

In [11]:
resid = 0.95

In [12]:
out = pagerank(adj_mat, n_iters=10000, resid=resid)

/tmp/ipykernel_33/2117850111.py:11: RuntimeWarning: divide by zero encountered in divide
  transitions = adjacency*(1/out_degrees[:, np.newaxis])


  0%|          | 0/10000 [00:00<?, ?it/s]

In [13]:
# get n closest papers using beam search
def get_n_closest_papers_via_beam_search(n, n_beams, transition_matrix, stationary_probability, graph, paper_id=None, paper_index=None, discount_factor=0.8, reverse_traversal=True):
    if paper_id is None and paper_index is None:
        raise Exception("paper_id and paper_index both cannot be None")
    if paper_id is not None:
        paper_index = graph.v_to_i[paper_id]
    if not reverse_traversal:
        transition_matrix = transition_matrix.T
        
    # set with all the best papers
    best_papers_set = {}
    # set the input index as the current index
    curr_paper_index = paper_index
    # number of papers explored
    papers_explored = 0
    
    # run until we have the required number of papers
    while len(best_papers_set) <= n:
        # set containing papers from the current beam
        beam_set = {}
        # calculate the probs to the next paper taking a col (row in this implementation) from the transpose of the matrix
        probs = (discount_factor**(len(best_papers_set)//n_beams))*(resid*transition_matrix[curr_paper_index:curr_paper_index+1, :].toarray() + (1 - resid)*(1/n_nodes))
        # multiply the markov jump probability with the page rank
        probs_real = np.squeeze(probs)*stationary_probability
        # get the indices with the highes probabilities
        ind = np.argpartition(probs_real, -100)
        ind = ind[np.argsort(probs_real[ind])]
        
        # add papers till the beam set is full starting from the last paper
        curr_ind = -1
        while len(beam_set) <= n_beams:
            if ind[curr_ind] not in best_papers_set and ind[curr_ind] != curr_paper_index:
                beam_set[ind[curr_ind]] = probs_real[ind[curr_ind]]
            curr_ind -= 1
        
        # add beam set to the set to be returned
        keys = list(beam_set.keys())
        keys.reverse()
        best_papers_set = {**best_papers_set, **beam_set}
        curr_paper_index = list(best_papers_set.keys())[papers_explored]
        papers_explored += 1
        
    return best_papers_set

def get_n_best_papers_page_rank(n, n_beams, transition_matrix, stationary_probability, graph, paper_id=None, paper_index=None, discount_factor=0.8, bwd_percent=0.8):
    fwd_papers = get_n_closest_papers_via_beam_search(int(n*bwd_percent), n_beams, transition_matrix, stationary_probability, graph, paper_id, paper_index, discount_factor, reverse_traversal=True)
    bwd_papers = get_n_closest_papers_via_beam_search(n - int(n*bwd_percent), n_beams, transition_matrix, stationary_probability, graph, paper_id, paper_index, discount_factor, reverse_traversal=False)
    
    return {**fwd_papers, **bwd_papers}



# get_n_best_papers(n=10, n_beams=2, transition_matrix, stationary_probability, graph, paper_id=None, paper_index=None, discount_factor=0.8, reverse_traversal=True)

In [14]:
papers_read = ['1512.03385', '1706.03762', '1312.5602']

closest_papers = {}
for paper_id in papers_read:
    closest_papers = {**closest_papers, **get_n_best_papers_page_rank(n=5, n_beams=3, transition_matrix=out[1], stationary_probability=np.squeeze(out[0]), graph=graph, paper_id=paper_id)}

In [15]:
emb_df = pd.read_parquet('/kaggle/input/dataset-with-embeddings/embeddings.parquet')

In [16]:
def get_cosine_scores(embed_df, papers_read):
    norm_embeds = np.array(embed_df)/np.linalg.norm(np.array(embed_df), axis=0)
    
    embedding = np.array(embed_df[df['id'].isin(papers_read)])
    norm_paper_embeds = (embedding/np.linalg.norm(embedding, axis=0))
    
    sim_df = norm_embeds.dot(norm_paper_embeds.T)
    sim_df = sim_df.max(axis=1)
    
    return sim_df

In [17]:
sim_df = get_cosine_scores(emb_df, papers_read)

In [18]:
def get_final_recs(closest_papers, sim_df, alpha=0.8):
    normalised_probs = np.array(list(closest_papers.values()))
    normalised_probs = normalised_probs/normalised_probs.sum()
    
    best_sim_df = sim_df
    best_sim_df[list(closest_papers.keys())] = best_sim_df[list(closest_papers.keys())]/best_sim_df[list(closest_papers.keys())].sum()
    
    final_papers = {}
    for id, paper in enumerate(closest_papers):
    #     selected_paper_id = list(closest_papers.keys())[paper]
        selected_paper_prob = alpha * normalised_probs[id] + (1 - alpha) * best_sim_df[paper]
        final_papers[paper] = selected_paper_prob
        
    best_paper_indices = np.flip(np.argsort(np.array(list(final_papers.values()))))
    
    return final_papers, best_paper_indices

In [19]:
final_papers, best_paper_indices = get_final_recs(closest_papers, sim_df, 0.5)

In [20]:
for id in best_paper_indices:
    paper_index = list(final_papers.keys())[id]
    df_row=df.iloc[paper_index]
    print(df_row['id'])
    print(df_row["title"])
    print(df_row["abstract"])
    print("Citations", df_row["num_citations"])
    print("References", df_row["num_references"])
    print("Degree", df_row["degree"])
    print()

1408.5093
Caffe: Convolutional Architecture for Fast Feature Embedding
  Caffe provides multimedia scientists and practitioners with a clean and modifiable framework for state-of-the-art deep learning algorithms and a collection of reference models. The framework is a BSD-licensed C++ library with Python and MATLAB bindings for training and deploying general-purpose convolutional neural networks and other deep models efficiently on commodity architectures. Caffe fits industry and internet-scale media needs by CUDA GPU computation, processing over 40 million images a day on a single K40 or Titan GPU ($\approx$ 2.5 ms per image). By separating model representation from actual implementation, Caffe allows experimentation and seamless switching among platforms for ease of development and deployment from prototyping machines to cloud environments. Caffe is maintained and developed by the Berkeley Vision and Learning Center (BVLC) with the help of an active community of contributors on GitHu